(a) What Unicode character does chr(0) return?

Deliverable: An empty string which also marks the end of a string
(b) How does this character’s string representation (__repr__()) differ from its printed representation?
Deliverable: A one-sentence response.
(c) What happens when this character occurs in text? It may be helpful to play around with the
following in your Python interpreter and see if it matches your expectations:
>>> chr(0)
>>> print(chr(0))
>>> "this is a test" + chr(0) + "string"
>>> print("this is a test" + chr(0) + "string")

In [16]:
chr(0)
print(chr(0))
# str(chr(0))
chr(0).__repr__()
chr(0).__str__()
# print(str('This is a test '+chr(0)+" string"))
# print("This is a test "+chr(0)+" string")

 


'\x00'

In [21]:
test_str = "你好, Hello"
utf8_bytes = test_str.encode('utf-8')
print(utf8_bytes)
print(type(utf8_bytes))
list_of_bytes = list(utf8_bytes)
print(list_of_bytes)
print(len(list_of_bytes))
print(len(test_str))
print(utf8_bytes.decode('utf-8'))

b'\xe4\xbd\xa0\xe5\xa5\xbd, Hello'
<class 'bytes'>
[228, 189, 160, 229, 165, 189, 44, 32, 72, 101, 108, 108, 111]
13
9
你好, Hello


In [26]:
utf32_bytes = test_str.encode('utf-32')
print(utf32_bytes)
print(list(utf32_bytes))
print(len(list(utf32_bytes)))

b'\xff\xfe\x00\x00`O\x00\x00}Y\x00\x00,\x00\x00\x00 \x00\x00\x00H\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00'
[255, 254, 0, 0, 96, 79, 0, 0, 125, 89, 0, 0, 44, 0, 0, 0, 32, 0, 0, 0, 72, 0, 0, 0, 101, 0, 0, 0, 108, 0, 0, 0, 108, 0, 0, 0, 111, 0, 0, 0]
40


In [34]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
byte_ = '你好'.encode('utf-8')
decode_utf8_bytes_to_str_wrong(byte_)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 0: unexpected end of data

Non-ASCII code (more than 1 byte) should not be decoded in this way.

In [50]:
seq = [0b11000000,0b11000000]
# convert seq to bytes
bytes(seq).decode('utf-8')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte

reason for failure: For two byte chraracter in UTF-8, it must be 0b110XXXXX 0b10XXXXXX.

In [4]:
pat_str=r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [1]:
import regex as re

In [5]:
re.findall(pat_str, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

# Trainable parameters in GPT-2 XL:

- Embedding : vocab_size $\times$ d_model = 50,257 $\times$ 1,600 = 80411200
- 48 transformer block: 
    - 2 RMS norm: 2 $\times$ d_model = 2 $\times$ 1,600 = 3200
    - 1 multiheadselfattention
        - 4 embedding matrix WQ WK WV WO: 4 $\times$ d_model $\times$ d_model = 4 $\times$ 1600 $\times$ 1600 = 10240000
    - FFN: SwiGLU: 3 $\times$ d_ff $\times$ d_model = 3 $\times$ 6400 $\times$ 1600 = 30720000
48 $\times$ (30720000+10240000+3200) = 1966233600
- final layers: 1600 + 80411200 = 80412800

Total:
80411200 + 1966233600 + 80412800 = 2,127,057,600 learnable parameters

when using FP32, it requires 2,127,057,600 $\times$ 32/8 / $2^30$ = 7.92 GB memory

# Matrix multiplies and FLOPS of GPT-2 XL
- Embedding: No matrix multiplication
- 48 transformer block:
    - Q K V projection:3 $\times$ 2 seq_length $\times$ d_model $\times$ seq_length
    - RoPE for Q K : 4 $\times$ seq_length $\times$ d_model
    - attention : 4 seq_length $\times$ d_model $\times$ seq_length
    - FFN : 6 $\times$ d_ff $\times$ d_model $\times$ seq_length
- Final linear projection: 2 seq_length $\times$ d_model $\times$ vocab_size

Total leading order part:

48 ($\times$ 10 d_model $\times$ (seq_length)$^2$ + 6 $\times$ d_ff $\times$ d_model $\times$ seq_length) + 2 seq_length $\times$ d_model $\times$ vocab_size = 48 $\times$ (16777216000 + 62914560000) + 164682137600 = 3989887385600 FLOPs= 3.715 TFLOPs

In [6]:
def Gflops(layer,d_model,seq_length = 1024,vocab_size = 50257):
    return (layer * (10 * d_model * seq_length**2 + 6 * 4 * d_model**2 * seq_length) + 2* seq_length * d_model * vocab_size)/2**30
print("Small:",Gflops(12,768),'GFLOPs')
print("Med:",Gflops(16,1024),'GFLOPs')
print("large:",Gflops(36,1280),'GFLOPs')
print("XL:",Gflops(48,1600),'GFLOPs')
print("XL-context length to 16384:",Gflops(48,1600,seq_length = 16384),'GFLOPs')

Small: 325.61865234375 GFLOPs
Med: 642.158203125 GFLOPs
large: 1922.69775390625 GFLOPs
XL: 3715.8721923828125 GFLOPs
XL-context length to 16384: 239453.955078125 GFLOPs


# Resource counting for training using AdamW

vocab_size, context_length,
num_layers, d_model, num_heads, d_ff = 4 d_model

In [14]:
def memory_consumption_Adam(vocab_size = 50257, context_length  = 1024,num_layers = 48, d_model = 1600, num_heads = 25, batch_size = 1):
    d_ff = 4 * d_model
    total_memory_consumption_float = 0
    # weights:
    # embedding
    embedding_memory = vocab_size * d_model
    # transformer
    RMS_norm_memory = 2 * d_model
    KQVO = 4 * d_model ** 2
    FFN = 2 * d_model * d_ff
    transformer_memory_per_layer = RMS_norm_memory + KQVO + FFN
    transformer_memory_total = num_layers * transformer_memory_per_layer
    # output
    embedding_memory_out = vocab_size * d_model +d_model
    total_LLM_weights_memory = embedding_memory + transformer_memory_total + embedding_memory_out

    # Activations :
    RMS_block = 2 * context_length * d_model
    KQV =  context_length * d_model
    KQ = context_length ** 2
    softmax = context_length ** 2
    K = context_length ** 2
    Q = context_length ** 2

    FFN = 3 * d_model * d_ff  +  context_length * d_model

    transformers = (RMS_block+KQV+KQ+softmax+K+Q) * num_layers
    Final_RMS = context_length * d_model
    Final_embedding = context_length * d_model
    activations = (transformers + Final_RMS + Final_embedding) * batch_size



    # Gradients and AdamW status
    gradients = total_LLM_weights_memory
    Adamstats = 2 * total_LLM_weights_memory # m and v
    optimizer_memory = gradients + Adamstats

    # print(total_LLM_weights_memory/2**30)
    # print(activations/2**30)
    # print(optimizer_memory/2**30)
    all_mem = (total_LLM_weights_memory + activations + optimizer_memory)/2**30
    # print(all_mem)
    return all_mem*4

print(memory_consumption_Adam())
print(memory_consumption_Adam(batch_size = 34))

26.01252269744873
80.16926097869873


# For batch = 1, flops = 3.71 * 3 = 11.129 TFLOPS
# For batch = 1024 ;flops = 11397 Flops
# for 400 K round, 400000*11397/(19.5/2) = 467574153s = 5411 days